In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input

In [3]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1000, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1000,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)
        y_batch = np.eye(2)[self.y[batch_indices]]
        return X_batch, y_batch
    
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")
        
class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        """
        Early stopping within the same epoch based on a monitored metric.
        Args:
            monitor: Metric to monitor ('loss', 'accuracy', etc.).
            threshold: Threshold value for stopping (e.g., loss < 0.1).
            patience: Number of batches to wait for improvement before stopping.
        """
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0 

batch_early_stopping_callback = BatchEarlyStopping(
    monitor='loss',     
    threshold=0.1,       
    patience=2          
)

def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  

    sequence_input = Input(shape=(seq_length, height, width, channels))
    
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128, return_sequences=True)(flattened_features)  # Output is 3D
    lstm_out = Dropout(0.4)(lstm_out)
    lstm_out = LSTM(128)(lstm_out) 
    lstm_out = Dropout(0.3)(lstm_out)


    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.3)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

indices = np.arange(1000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', train_indices, batch_size=10)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', val_indices, batch_size=10)

checkpoint_dir = './Further_improvements_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',  
    save_best_only=False,
    save_weights_only=False,  
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2: 
        return lr
    return lr * 0.9 

lr_callback = LearningRateScheduler(scheduler)

step_timer_callback = StepTimerCallback()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1  
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 15, 128)        │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,124,994 (111.10 MB)

 Trainable params: 25,896,130 (98.79 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/10
Step 1/80 - Loss: 0.6816, Accuracy: 0.5000, Time: 89.03 seconds
 1/80 ━━━━━━━━━━━━━━━━━━━━ 2:04:48 95s/step - accuracy: 0.5000 - loss: 0.6816Step 2/80 - Loss: 0.6892, Accuracy: 0.4000, Time: 3.61 seconds
 2/80 ━━━━━━━━━━━━━━━━━━━━ 4:51 4s/step - accuracy: 0.4500 - loss: 0.6854    Step 3/80 - Loss: 0.6889, Accuracy: 0.4333, Time: 3.36 seconds
 3/80 ━━━━━━━━━━━━━━━━━━━━ 4:33 4s/step - accuracy: 0.4444 - loss: 0.6866Step 4/80 - Loss: 0.6988, Accuracy: 0.4250, Time: 3.27 seconds
 4/80 ━━━━━━━━━━━━━━━━━━━━ 4:22 3s/step - accuracy: 0.4396 - loss: 0.6896Step 5/80 - Loss: 0.7073, Accuracy: 0.4400, Time: 3.20 seconds
 5/80 ━━━━━━━━━━━━━━━━━━━━ 4:14 3s/step - accuracy: 0.4397 - loss: 0.6932Step 6/80 - Loss: 0.7094, Accuracy: 0.4500, Time: 3.01 seconds
 6/80 ━━━━━━━━━━━━━━━━━━━━ 4:05 3s/step - accuracy: 0.4414 - loss: 0.6959Step 7/80 - Loss: 0.7114, Accuracy: 0.4571, Time: 3.37 seconds
 7/80 ━━━━━━━━━━━━━━━━━━━━ 4:02 3s/step - accuracy: 0.4436 - loss: 0.6981St

In [4]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.7750 - loss: 0.4870
Validation Loss: 0.5052
Validation Accuracy: 0.7600


In [5]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

20/20 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.73      0.84      0.78       103
     Class 1       0.80      0.67      0.73        97

    accuracy                           0.76       200
   macro avg       0.77      0.76      0.76       200
weighted avg       0.77      0.76      0.76       200

Confusion Matrix:
[[87 16]
 [32 65]]


In [12]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras.layers import Input, Dense, LSTM, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
from keras.utils import Sequence
from sklearn.model_selection import train_test_split
import numpy as np
import os
import time

class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1000, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1000,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  
        y_batch = np.eye(2)[self.y[batch_indices]]  
        return X_batch, y_batch

class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None and current_value < self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                self.model.stop_training = True
        else:
            self.wait = 0

batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

checkpoint_dir = './Further_improvements_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    return lr * 0.9

lr_callback = LearningRateScheduler(scheduler)

indices = np.arange(1000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', train_indices, batch_size=10)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', val_indices, batch_size=10)

latest_checkpoint = r'E:\PosePerfect\Model\Further_improvements_Checkpoints\model_epoch_10_val_loss_0.51.keras'
cnn_lstm_model = load_model(latest_checkpoint)

cnn_lstm_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

cnn_lstm_model.summary()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5, 
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, StepTimerCallback(), batch_early_stopping_callback],
    verbose=1
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 15, 128)        │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,124,994 (111.10 MB)

 Trainable params: 25,896,130 (98.79 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/5
Step 1/80 - Loss: 0.3186, Accuracy: 1.0000, Time: 91.62 seconds
 1/80 ━━━━━━━━━━━━━━━━━━━━ 2:11:24 100s/step - accuracy: 1.0000 - loss: 0.3186Step 2/80 - Loss: 0.3363, Accuracy: 0.9500, Time: 4.02 seconds
 2/80 ━━━━━━━━━━━━━━━━━━━━ 5:53 5s/step - accuracy: 0.9750 - loss: 0.3275     Step 3/80 - Loss: 0.3559, Accuracy: 0.9333, Time: 2.85 seconds
 3/80 ━━━━━━━━━━━━━━━━━━━━ 4:44 4s/step - accuracy: 0.9611 - loss: 0.3369Step 4/80 - Loss: 0.4975, Accuracy: 0.8000, Time: 3.64 seconds
 4/80 ━━━━━━━━━━━━━━━━━━━━ 4:39 4s/step - accuracy: 0.9208 - loss: 0.3771Step 5/80 - Loss: 0.5388, Accuracy: 0.7600, Time: 2.69 seconds
 5/80 ━━━━━━━━━━━━━━━━━━━━ 4:17 3s/step - accuracy: 0.8887 - loss: 0.4094Step 6/80 - Loss: 0.5491, Accuracy: 0.7500, Time: 2.72 seconds
 6/80 ━━━━━━━━━━━━━━━━━━━━ 4:03 3s/step - accuracy: 0.8656 - loss: 0.4327Step 7/80 - Loss: 0.5296, Accuracy: 0.7714, Time: 2.67 seconds
 7/80 ━━━━━━━━━━━━━━━━━━━━ 3:52 3s/step - accuracy: 0.8521 - loss: 0.4465S

In [13]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.7797 - loss: 0.4196
Validation Loss: 0.4414
Validation Accuracy: 0.7750


In [14]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

20/20 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.75      0.84      0.79       103
     Class 1       0.81      0.70      0.75        97

    accuracy                           0.78       200
   macro avg       0.78      0.77      0.77       200
weighted avg       0.78      0.78      0.77       200

Confusion Matrix:
[[87 16]
 [29 68]]


In [2]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras.layers import Input, Dense, LSTM, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
from keras.utils import Sequence
from sklearn.model_selection import train_test_split
import numpy as np
import os
import time

class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1000, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1000,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  
        y_batch = np.eye(2)[self.y[batch_indices]]  
        return X_batch, y_batch

class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None and current_value < self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                self.model.stop_training = True
        else:
            self.wait = 0

batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

checkpoint_dir = './Further_improvements_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    return lr * 0.9

lr_callback = LearningRateScheduler(scheduler)

indices = np.arange(1000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', train_indices, batch_size=8)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', val_indices, batch_size=8)

latest_checkpoint = r'E:\PosePerfect\Model\Further_improvements_Checkpoints\model_epoch_05_val_loss_0.44.keras'
cnn_lstm_model = load_model(latest_checkpoint)

cnn_lstm_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

cnn_lstm_model.summary()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5, 
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, StepTimerCallback(), batch_early_stopping_callback],
    verbose=1
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 7, 7, 1024) │     3,228,864 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 50176)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 15, 128)        │    25,756,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,124,994 (111.10 MB)

 Trainable params: 25,896,130 (98.79 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

E:\PosePerfect\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



--- Starting Epoch 1 ---
Epoch 1/5
Step 1/100 - Loss: 0.1903, Accuracy: 1.0000, Time: 81.33 seconds
  1/100 ━━━━━━━━━━━━━━━━━━━━ 2:18:37 84s/step - accuracy: 1.0000 - loss: 0.1903Step 2/100 - Loss: 0.3449, Accuracy: 0.8750, Time: 2.76 seconds
  2/100 ━━━━━━━━━━━━━━━━━━━━ 4:44 3s/step - accuracy: 0.9375 - loss: 0.2676    Step 3/100 - Loss: 0.3033, Accuracy: 0.9167, Time: 2.57 seconds
  3/100 ━━━━━━━━━━━━━━━━━━━━ 4:25 3s/step - accuracy: 0.9306 - loss: 0.2795Step 4/100 - Loss: 0.2682, Accuracy: 0.9375, Time: 2.54 seconds
  4/100 ━━━━━━━━━━━━━━━━━━━━ 4:16 3s/step - accuracy: 0.9323 - loss: 0.2767Step 5/100 - Loss: 0.2822, Accuracy: 0.9250, Time: 2.43 seconds
  5/100 ━━━━━━━━━━━━━━━━━━━━ 4:08 3s/step - accuracy: 0.9308 - loss: 0.2778Step 6/100 - Loss: 0.3283, Accuracy: 0.8958, Time: 2.49 seconds
  6/100 ━━━━━━━━━━━━━━━━━━━━ 4:03 3s/step - accuracy: 0.9250 - loss: 0.2862Step 7/100 - Loss: 0.3646, Accuracy: 0.8929, Time: 2.40 seconds
  7/100 ━━━━━━━━━━━━━━━━━━━━ 3:58 3s/step - accuracy: 0.9

In [3]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.8456 - loss: 0.3564
Validation Loss: 0.3701
Validation Accuracy: 0.8500


In [4]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

25/25 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.97      0.73      0.83       103
     Class 1       0.77      0.98      0.86        97

    accuracy                           0.85       200
   macro avg       0.87      0.85      0.85       200
weighted avg       0.88      0.85      0.85       200

Confusion Matrix:
[[75 28]
 [ 2 95]]


In [1]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\Further_improvements_Checkpoints\model_epoch_05_val_loss_0.37.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

Model loaded successfully!


In [3]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_biceps_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-0.9997539520263672, Max=-0.9921568632125854, Mean=-0.9957398772239685
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step
Raw Predictions: [[0.9491511  0.0508489 ]
 [0.9486577  0.05134236]]
Predicted Class Probabilities: [0.9491511 0.0508489]
Predicted Class: [0 0]


In [4]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_squats_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-0.9997539520263672, Max=-0.9921568632125854, Mean=-0.9958739876747131
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Raw Predictions: [[0.75177497 0.24822503]
 [0.72622675 0.27377325]]
Predicted Class Probabilities: [0.75177497 0.24822503]
Predicted Class: [0 0]


In [5]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_7.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9972225427627563
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 843ms/step
Raw Predictions: [[0.8971972  0.10280282]
 [0.8376328  0.16236724]]
Predicted Class Probabilities: [0.8971972  0.10280282]
Predicted Class: [0 0]


In [6]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.9970716238021851
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
Raw Predictions: [[0.06328028 0.9367197 ]
 [0.08500309 0.914997  ]]
Predicted Class Probabilities: [0.06328028 0.9367197 ]
Predicted Class: [1 1]


In [2]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_biceps_2.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-0.9998462200164795, Max=-0.9921568632125854, Mean=-0.9950932860374451
1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step
Raw Predictions: [[0.90659523 0.0934048 ]
 [0.91572094 0.08427905]]
Predicted Class Probabilities: [0.90659523 0.0934048 ]
Predicted Class: [0 0]


In [3]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_squats_2.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

Input preprocessed successfully!
Input Shape: (2, 15, 224, 224, 3)
Input Summary: Min=-1.0, Max=-0.9921568632125854, Mean=-0.996332049369812
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw Predictions: [[0.52333415 0.47666582]
 [0.5360343  0.4639657 ]]
Predicted Class Probabilities: [0.52333415 0.47666582]
Predicted Class: [0 0]
